LAB 9

In [1]:
pip install scikit-learn pandas numpy

In [10]:

# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score, classification_report
import re
from nltk.corpus import stopwords
import nltk
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.datasets import load_iris
import re
from nltk.corpus import stopwords


In [4]:

# Download NLTK stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
# Load stopwords (assuming NLTK is pre-installed in Colab; fallback if not found)
try:
    stop_words = set(stopwords.words('english'))
except LookupError:
    import nltk
    nltk.download('stopwords')
    stop_words = set(stopwords.words('english'))

# Upload the dataset in Colab (run this cell first to upload the file)
from google.colab import files
uploaded = files.upload()  # This will prompt you to upload SMSSpamCollection

Saving SMSSpamCollection to SMSSpamCollection


In [6]:
# Task 1: Load and Preprocess the Dataset
def load_and_preprocess_data(file_path):
    # Load dataset
    df = pd.read_csv(file_path, sep='\t', names=['label', 'message'])

    # Convert labels to binary (ham=0, spam=1)
    df['label'] = df['label'].map({'ham': 0, 'spam': 1})

    # Text preprocessing function
    stop_words = set(stopwords.words('english'))

    def preprocess_text(text):
        # Lowercase
        text = text.lower()
        # Remove punctuation and special characters
        text = re.sub(r'[^\w\s]', '', text)
        # Tokenize and remove stopwords
        tokens = [word for word in text.split() if word not in stop_words]
        return ' '.join(tokens)

    # Apply preprocessing
    df['message'] = df['message'].apply(preprocess_text)

    return df

In [7]:
# Task 2: Train Multinomial Naive Bayes
def train_multinomial_nb(X, y):
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Convert text to TF-IDF features
    vectorizer = TfidfVectorizer()
    X_train_tfidf = vectorizer.fit_transform(X_train)
    X_test_tfidf = vectorizer.transform(X_test)

    # Train model
    mnb = MultinomialNB()
    mnb.fit(X_train_tfidf, y_train)

    # Predict and evaluate
    y_pred = mnb.predict(X_test_tfidf)
    print("\nMultinomial Naive Bayes Results:")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(classification_report(y_test, y_pred, target_names=['ham', 'spam']))

    return mnb, vectorizer

In [8]:
# Task 3: Train Bernoulli Naive Bayes
def train_bernoulli_nb(X, y):
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Convert text to TF-IDF features (binary for Bernoulli)
    vectorizer = TfidfVectorizer(binary=True)
    X_train_tfidf = vectorizer.fit_transform(X_train)
    X_test_tfidf = vectorizer.transform(X_test)

    # Train model
    bnb = BernoulliNB()
    bnb.fit(X_train_tfidf, y_train)

    # Predict and evaluate
    y_pred = bnb.predict(X_test_tfidf)
    print("\nBernoulli Naive Bayes Results:")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(classification_report(y_test, y_pred, target_names=['ham', 'spam']))

In [9]:
# Main execution
if __name__ == "__main__":
    # Load and preprocess SMS Spam dataset
    file_path = "SMSSpamCollection"  # Update this path if needed
    df = load_and_preprocess_data(file_path)
    X, y = df['message'], df['label']

    # Run Naive Bayes experiments
    mnb_model, vectorizer = train_multinomial_nb(X, y)
    train_bernoulli_nb(X, y)


Multinomial Naive Bayes Results:
Accuracy: 0.9704
              precision    recall  f1-score   support

         ham       0.97      1.00      0.98       966
        spam       1.00      0.78      0.88       149

    accuracy                           0.97      1115
   macro avg       0.98      0.89      0.93      1115
weighted avg       0.97      0.97      0.97      1115


Bernoulli Naive Bayes Results:
Accuracy: 0.9749
              precision    recall  f1-score   support

         ham       0.97      1.00      0.99       966
        spam       0.99      0.82      0.90       149

    accuracy                           0.97      1115
   macro avg       0.98      0.91      0.94      1115
weighted avg       0.98      0.97      0.97      1115



#LAB 10


In [11]:
try:
    stop_words = set(stopwords.words('english'))
except LookupError:
    import nltk
    nltk.download('stopwords')
    stop_words = set(stopwords.words('english'))

# Upload the dataset in Colab (run this cell first to upload the file)
from google.colab import files
uploaded = files.upload()

Saving SMSSpamCollection to SMSSpamCollection (1)


In [12]:
def load_and_preprocess_data(file_path='SMSSpamCollection'):
    df = pd.read_csv(file_path, sep='\t', names=['label', 'message'])
    df['label'] = df['label'].map({'ham': 0, 'spam': 1})

    def preprocess_text(text):
        text = text.lower()
        text = re.sub(r'[^\w\s]', '', text)
        tokens = [word for word in text.split() if word not in stop_words]
        return ' '.join(tokens)

    df['message'] = df['message'].apply(preprocess_text)
    return df

In [13]:
def kfold_iris():
    iris = load_iris()
    X, y = iris.data, iris.target

    gnb = GaussianNB()
    scores = cross_val_score(gnb, X, y, cv=5)

    print("\nK-Fold Cross Validation (Iris - Gaussian NB):")
    print(f"Accuracy scores for each fold: {scores}")
    print(f"Mean accuracy: {scores.mean():.4f} (+/- {scores.std() * 2:.4f})")

In [14]:
def stratified_kfold_sms(X, y):
    vectorizer = TfidfVectorizer()
    X_tfidf = vectorizer.fit_transform(X)

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    mnb = MultinomialNB()

    print("\nStratified K-Fold Cross Validation (SMS Spam):")
    fold = 1
    for train_idx, test_idx in skf.split(X_tfidf, y):
        X_train, X_test = X_tfidf[train_idx], X_tfidf[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        mnb.fit(X_train, y_train)
        score = mnb.score(X_test, y_test)
        print(f"Fold {fold} - Accuracy: {score:.4f}")
        print(f"Class distribution in test set: {np.bincount(y_test)}")
        fold += 1

In [17]:
def compare_split_vs_kfold(X, y):
    vectorizer = TfidfVectorizer()
    X_tfidf = vectorizer.fit_transform(X)

    # Single train-test split
    X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)
    mnb = MultinomialNB()
    mnb.fit(X_train, y_train)
    split_score = mnb.score(X_test, y_test)

    # K-Fold
    kfold_scores = cross_val_score(mnb, X_tfidf, y, cv=5)

    print("\nTrain-Test Split vs K-Fold Comparison (SMS Spam):")
    print(f"Single Split Accuracy: {split_score:.4f}")
    print(f"K-Fold Mean Accuracy: {kfold_scores.mean():.4f} (+/- {kfold_scores.std() * 2:.4f})")

In [18]:
if __name__ == "__main__":
    # Load and preprocess the uploaded SMS Spam dataset
    df = load_and_preprocess_data('SMSSpamCollection')
    X, y = df['message'], df['label']

    # Run Cross Validation experiments
    kfold_iris()
    stratified_kfold_sms(X, y)
    compare_split_vs_kfold(X, y)


K-Fold Cross Validation (Iris - Gaussian NB):
Accuracy scores for each fold: [0.93333333 0.96666667 0.93333333 0.93333333 1.        ]
Mean accuracy: 0.9533 (+/- 0.0533)

Stratified K-Fold Cross Validation (SMS Spam):
Fold 1 - Accuracy: 0.9659
Class distribution in test set: [965 150]
Fold 2 - Accuracy: 0.9623
Class distribution in test set: [965 150]
Fold 3 - Accuracy: 0.9614
Class distribution in test set: [965 149]
Fold 4 - Accuracy: 0.9605
Class distribution in test set: [965 149]
Fold 5 - Accuracy: 0.9650
Class distribution in test set: [965 149]

Train-Test Split vs K-Fold Comparison (SMS Spam):
Single Split Accuracy: 0.9704
K-Fold Mean Accuracy: 0.9618 (+/- 0.0086)
